<a href="https://colab.research.google.com/github/paulynamagana/Covid19_Pneumonia_Classification/blob/main/classification_covid_pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>